# Intro

- Building a GPT from scratch, following Andrej Karpathy's lesson at: https://youtu.be/kCc8FmEb1nY?si=4AZNnwmnb4XflQT8
- Character level model, trained on Shakespears works

In [2]:
# clone repo for input text files
!git clone https://github.com/karpathy/ng-video-lecture.git

Cloning into 'ng-video-lecture'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 64 (delta 22), reused 18 (delta 18), pack-reused 32 (from 1)
Receiving objects: 100% (64/64), 441.23 KiB | 11.92 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [3]:
# open file we will use to train on
file_path = '/kaggle/working/ng-video-lecture/input.txt'
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("Total chracters in data set:", len(text))

Total chracters in data set: 1115394


In [5]:
# some samples from the training set
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
# all unique characters
chars = set(text)
vocab_size = len(chars)
print(f"{vocab_size} char set:", ''.join(sorted(list(chars))))

65 char set: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [7]:
sorted_chars = sorted(list(chars))
for i in range(vocab_size):
    # checking space & new line characters
    c = sorted_chars[i]
    print(f"{i}: {ord(c)} '{c}'")

0: 10 '
'
1: 32 ' '
2: 33 '!'
3: 36 '$'
4: 38 '&'
5: 39 '''
6: 44 ','
7: 45 '-'
8: 46 '.'
9: 51 '3'
10: 58 ':'
11: 59 ';'
12: 63 '?'
13: 65 'A'
14: 66 'B'
15: 67 'C'
16: 68 'D'
17: 69 'E'
18: 70 'F'
19: 71 'G'
20: 72 'H'
21: 73 'I'
22: 74 'J'
23: 75 'K'
24: 76 'L'
25: 77 'M'
26: 78 'N'
27: 79 'O'
28: 80 'P'
29: 81 'Q'
30: 82 'R'
31: 83 'S'
32: 84 'T'
33: 85 'U'
34: 86 'V'
35: 87 'W'
36: 88 'X'
37: 89 'Y'
38: 90 'Z'
39: 97 'a'
40: 98 'b'
41: 99 'c'
42: 100 'd'
43: 101 'e'
44: 102 'f'
45: 103 'g'
46: 104 'h'
47: 105 'i'
48: 106 'j'
49: 107 'k'
50: 108 'l'
51: 109 'm'
52: 110 'n'
53: 111 'o'
54: 112 'p'
55: 113 'q'
56: 114 'r'
57: 115 's'
58: 116 't'
59: 117 'u'
60: 118 'v'
61: 119 'w'
62: 120 'x'
63: 121 'y'
64: 122 'z'


In [8]:
# Encoder/Decoder: each character maps to an integer which its the indx in the sorted array
c_to_i = {c:i for i,c in enumerate(sorted_chars)}
i_to_c = {i:c for i,c in enumerate(sorted_chars)}

def encode(s):
    # takes input string to encode
    return [c_to_i[c] for c in s]


def decode(l):
    # takes an array of integers to decode
    return [i_to_c[i] for i in l]

In [9]:
print(encode("Hello World this is GPT"))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42, 1, 58, 46, 47, 57, 1, 47, 57, 1, 19, 28, 32]


In [10]:
x = encode("Hello World this is GPT")
print(decode(x))

['H', 'e', 'l', 'l', 'o', ' ', 'W', 'o', 'r', 'l', 'd', ' ', 't', 'h', 'i', 's', ' ', 'i', 's', ' ', 'G', 'P', 'T']


- Many other encoding schemas are possible, chat GPT and others mainly use **sub-word** tokenizers
- Here this simple model has a tradeoff that a small vocab size for the encoder == large vector encodings
- if we had a much larger vocab size == smaller vector encoding

# Pytorch
- We will use pytorch here to generate the data as a tensor for training

In [11]:
import torch
data = torch.tensor(encode(text)) # Tensor (m,) == 1D vector --> 1D array
print(data.shape, data.dtype)
print(data[:1000]) # 1000 characters encoded

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [12]:
# training and validation splits
n = int(0.9*len(data))
train = data[:n]
val = data[n:]

In [13]:
# we will train on chunks of the data
block_size = 8 # this will be the max context X to predict target Y
train[:block_size + 1] # we can train on 8 subarrays/instances and predict the next 1 given 9 total seq len

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [14]:
# example of a training instance
x = train[:block_size]
y = train[1:block_size + 1]

for i in range(block_size):
    context = x[:i+1] # given the context predict y
    target = y[i] # only predicts the next chracter in a given seq
    print("Context:", context, "Target:", target)

Context: tensor([18]) Target: tensor(47)
Context: tensor([18, 47]) Target: tensor(56)
Context: tensor([18, 47, 56]) Target: tensor(57)
Context: tensor([18, 47, 56, 57]) Target: tensor(58)
Context: tensor([18, 47, 56, 57, 58]) Target: tensor(1)
Context: tensor([18, 47, 56, 57, 58,  1]) Target: tensor(15)
Context: tensor([18, 47, 56, 57, 58,  1, 15]) Target: tensor(47)
Context: tensor([18, 47, 56, 57, 58,  1, 15, 47]) Target: tensor(58)


In [15]:
# create batch processing
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train if split == 'train' else val
    ix = torch.randint(len(data) - block_size, (batch_size,)) # outputs 1D vector of len == batch_size
    x = torch.stack([data[i:i+block_size] for i in ix]) # stacks the 1D vectors --> (4,8) 4 rows x 8 cols
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # (4,8) we will make 8 predictions y on each X
    return x, y

xb, yb = get_batch('train')
print(f"Inputs: {xb.shape} \n{xb} \nTargets: {yb.shape} \n{yb}")

Inputs: torch.Size([4, 8]) 
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]) 
Targets: torch.Size([4, 8]) 
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [16]:
# Total training samples per batch == batch_size * block_size == 8*4 == 32
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1] # sliding window context
        target = yb[b,t] # predict the next chracter c
        print(f"Context: {context} Target: {target}")

Context: tensor([24]) Target: 43
Context: tensor([24, 43]) Target: 58
Context: tensor([24, 43, 58]) Target: 5
Context: tensor([24, 43, 58,  5]) Target: 57
Context: tensor([24, 43, 58,  5, 57]) Target: 1
Context: tensor([24, 43, 58,  5, 57,  1]) Target: 46
Context: tensor([24, 43, 58,  5, 57,  1, 46]) Target: 43
Context: tensor([24, 43, 58,  5, 57,  1, 46, 43]) Target: 39
Context: tensor([44]) Target: 53
Context: tensor([44, 53]) Target: 56
Context: tensor([44, 53, 56]) Target: 1
Context: tensor([44, 53, 56,  1]) Target: 58
Context: tensor([44, 53, 56,  1, 58]) Target: 46
Context: tensor([44, 53, 56,  1, 58, 46]) Target: 39
Context: tensor([44, 53, 56,  1, 58, 46, 39]) Target: 58
Context: tensor([44, 53, 56,  1, 58, 46, 39, 58]) Target: 1
Context: tensor([52]) Target: 58
Context: tensor([52, 58]) Target: 1
Context: tensor([52, 58,  1]) Target: 58
Context: tensor([52, 58,  1, 58]) Target: 46
Context: tensor([52, 58,  1, 58, 46]) Target: 39
Context: tensor([52, 58,  1, 58, 46, 39]) Target

# Bigram Language Model
- Simplest possible NN model for LLM

In [18]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token reads off the logits for the next token from this lookup table of 65x65
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets):
        logits = self.token_embedding_table(idx) # (B,T,Channels=vocab_size) == (4,8,65)

        # reshape input tensor to match pytorch docs F.cross_entropy
        B, T, C = logits.shape
        logits = logits.view(B*T, C) # 2D tensor
        targets = targets.view(B*T) # 1D Tensor

        # calculate the loss
        loss = F.cross_entropy(logits, targets)
        return logits, loss

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb) # __call__ is enabled for the "forward" method of classes

print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [20]:
# Now lets see how this compares to the theoretical loss which is realted to entropy == number of states
import numpy as np

print(-np.log(1/65))

4.174387269895637


In [ ]:
# initial predictions are not that great due to thier entropy
# Here we will look at model generation

# min 37 (-10 mins or so)

# continuing here